In [33]:
#Load Data:
import pandas as pd
import os
data1g = pd.read_csv("data"+os.sep+"part"+os.sep+"Suomi112_cpd_google.csv", encoding='utf-8-sig')
data1g.drop(['Unnamed: 0','userName', 'reviewId', 'at','reviewCreatedVersion','score','thumbsUpCount'], axis=1, inplace=True)
data1a = pd.read_csv("data"+os.sep+"part"+os.sep+"Suomi112_cpd_apple.csv", encoding='utf-8-sig')
data1a.drop(['Unnamed: 0','title', 'date', 'rating'], axis=1, inplace=True)
data2g = pd.read_csv("data"+os.sep+"part"+os.sep+"SosLive_cpd_google.csv", encoding='utf-8-sig')
data2g.drop(['Unnamed: 0','userName', 'reviewId', 'at','reviewCreatedVersion','score','thumbsUpCount'], axis=1, inplace=True)
data2a = pd.read_csv("data"+os.sep+"part"+os.sep+"SosLive_cpd_apple.csv", encoding='utf-8-sig')
data2a.drop(['Unnamed: 0','title', 'date', 'rating','developerResponse'], axis=1, inplace=True)
data = pd.concat([data1g,data1g,data2g,data2a])

In [34]:
data

,content,sentiment
0,"The program is clear to use, only some informa...",Positive
1,The cerebral infarction patient was asked to v...,Negative
2,The 112 icon disappeared from the phone screen...,Positive
3,Positioning accuracy 1300m when I'm at home. 👎,Negative
4,A comfortable device,Negative
...,...,...
331,Good App but it is not accessible for the deaf.,Positive
332,Good app,Positive
333,"However, the app lacks a lock or similar so th...",Positive
334,Crazy that this didn't come up sooner. But bet...,Positive


In [35]:
sentiment_dict = \
{
    "Positive" : 1,
    "Negative" : 0,
    "Neutral"  : -1,
}
data['sentiment'] = [sentiment_dict[sent] for sent in data['sentiment']]
data = data.drop(data[data["sentiment"] == sentiment_dict["Neutral"]].index) # Remove Neutral Reviews
data

,content,sentiment
0,"The program is clear to use, only some informa...",1
1,The cerebral infarction patient was asked to v...,0
2,The 112 icon disappeared from the phone screen...,1
3,Positioning accuracy 1300m when I'm at home. 👎,0
6,Don't swim,0
...,...,...
330,It feels good that the app exists in case I am...,0
331,Good App but it is not accessible for the deaf.,1
333,"However, the app lacks a lock or similar so th...",1
334,Crazy that this didn't come up sooner. But bet...,1


In [36]:
import utils, importlib
importlib.reload(utils)
processor = utils.Processor()
processor.ini_dowload()
data_cpoy = data

[nltk_data] Downloading package stopwords to data/lib/stopwords...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to data/lib/wordnet...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to data/lib/omw...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [41]:
data['content'] = data['content'].apply(lambda x: processor.preprocess(str(x)))
data['content'] = data['content'].apply(lambda x: processor.tokenize(x))
data['content'] = data['content'].apply(lambda x: processor.remove_stopwords(x))
data['content'] = data['content'].apply(lambda x: processor.process_tokens(x))
data

,content,sentiment
0,"[program, clear, inform, resuscit, inform, var...",1
1,"[cerebr, infarct, patient, ask, verifi, phone,...",0
2,"[icon, disappear, phone, screen, although, ope...",1
3,"[posit, accuraci, home, thumb]",0
6,[swim],0
...,...,...
330,"[feel, good, exist, case, abus, someth, els, c...",0
331,"[good, access, deaf]",1
333,"[howev, lack, lock, similar, littl, kid, accid...",1
334,"[crazi, come, sooner, better, late, never]",1


In [155]:
data_cpoy = data

In [150]:
data = data_cpoy

In [151]:
from nltk.util import ngrams
import itertools
#Use Ngrams (both bigram and thrigrams):
data['ngrams'] = data['content'].apply(lambda x: list(itertools.chain(list(ngrams(x,2)),list(ngrams(x,3)))))
# Remove reviews with only one keyword:
data["ngrams"] = data["ngrams"].apply(lambda x: x if x != [] else "")
data = data[data.ngrams != ""]
data

,content,sentiment,ngrams
0,"[program, clear, inform, resuscit, inform, var...",1,"[(program, clear), (clear, inform), (inform, r..."
1,"[cerebr, infarct, patient, ask, verifi, phone,...",0,"[(cerebr, infarct), (infarct, patient), (patie..."
2,"[icon, disappear, phone, screen, although, ope...",1,"[(icon, disappear), (disappear, phone), (phone..."
3,"[posit, accuraci, home, thumb]",0,"[(posit, accuraci), (accuraci, home), (home, t..."
6,[swim],0,[]
...,...,...,...
330,"[feel, good, exist, case, abus, someth, els, c...",0,"[(feel, good), (good, exist), (exist, case), (..."
331,"[good, access, deaf]",1,"[(good, access), (access, deaf), (good, access..."
333,"[howev, lack, lock, similar, littl, kid, accid...",1,"[(howev, lack), (lack, lock), (lock, similar),..."
334,"[crazi, come, sooner, better, late, never]",1,"[(crazi, come), (come, sooner), (sooner, bette..."


In [ ]:
#random forest:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
from sklearn.ensemble import RandomForestClassifier